TODO add heder

## Setup

In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

In [13]:
!pip install gdown

inputs:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]

In [26]:
REGION = 'us-central1'
DATANAME = 'churn'
TABLE_NAME = 'churn_data'

In [23]:
SOURCE_FILE_NAME = 'churn_data.csv'

packages:

In [15]:
from google.cloud import aiplatform
import gdown

clients:

In [29]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project = PROJECT_ID)

parameters:

BUCKET_NAME = PROJECT_ID

# Download the dataset

To create a TabularDataset in Vertex AI from a CSV file, one first needs to load the CSV into a Google Cloud Storage bucket or into BigQuery.
After that, you we can create the dataset using the appropriate reference.
I chose to load the csv to BigQuery.

In [24]:
gdown.download(f"https://drive.google.com/uc?export=download&id=1YQbNdJNGgAp8z0W_BAtwoaQJ-_E2h1MX", SOURCE_FILE_NAME, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1YQbNdJNGgAp8z0W_BAtwoaQJ-_E2h1MX
To: /home/jupyter/churn_data.csv
100%|██████████| 978k/978k [00:00<00:00, 90.5MB/s]


'churn_data.csv'

# Load data to BigQuery

Create BigQuery Dataset

In [27]:
ds = bigquery.Dataset(f"{PROJECT_ID}.{DATANAME}")
ds.location = REGION
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [31]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
    autodetect=True,  # Let BigQuery autodetect the schema
)

# Load the data into BigQuery directly from the file
with open(SOURCE_FILE_NAME, "rb") as source_file:
    load_job = bq.load_table_from_file(
        source_file,
        ds.table(TABLE_NAME),
        job_config=job_config,
    )

load_job.result()  # Wait for the job to complete

print(f"Loaded {load_job.output_rows} rows into {DATANAME}.{TABLE_NAME}.")

Loaded 7043 rows into churn.churn_data.
